In [6]:
!pip install wget

  Using cached wget-3.2-py3-none-any.whl


In [7]:
import torch
import wget

In [3]:
batch_size = 4
block_size = 8

torch.randint(1000000- block_size, (batch_size,))  #randint(low, (size=4,1))

tensor([617986,  27814, 356481, 870909])

In [9]:
file = wget.download('https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt')

100% [..........................................................................] 1115394 / 1115394

In [10]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [11]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [12]:
chars = sorted(list(set(text)))
vocab_size = len(chars)

In [25]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda s: ''.join(itos[c] for c in s)

In [27]:
data = torch.tensor(encode(text), dtype=torch.long)
n = int(len(data)*0.8)
train_data = data[:n]
test_data = data[n:]

In [ ]:
def get_batch():
        